# Assignment-5-part-b- TimeGPT Fine-tuning

## Assignment Description
Fine-tune TimeGPT using your own data and explain the finetuning process step-by-step within the Colab.
Hint: Refer to Nixtla's fine-tuning tutorial.




In [1]:
# Ensure that the necessary libraries for TimeGPT fine-tuning are installed.
!pip install -Uqq nixtla

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.2 MB/s eta 0:00:00


In [2]:
# Import libraries for handling data and working with the TimeGPT model.
import pandas as pd
from nixtla import NixtlaClient
from utilsforecast.losses import mae, mse
from utilsforecast.evaluation import evaluate

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from google.colab import userdata
NIXTLA_API_KEY = userdata.get('NIXTLA_API_KEY')

In [4]:
# Initialize the Nixtla client with the API key (replace with your actual API key)
nixtla_client = NixtlaClient(api_key=NIXTLA_API_KEY)

In [11]:
# Load a time-series dataset (use your own or an example dataset)
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/air_passengers.csv')
print("Dataset Preview:")
print(df.head())

Dataset Preview:
    timestamp  value
0  1949-01-01    112
1  1949-02-01    118
2  1949-03-01    132
3  1949-04-01    129
4  1949-05-01    121


In [18]:
import pandas as pd
import nixtla
import matplotlib.pyplot as plt

# Load your dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')

# Renaming columns for consistency
df.rename(columns={'Month': 'timestamp', 'Sunspots': 'value'}, inplace=True)

# Checking the first few rows of the dataset
print("Dataset Preview:")
print(df.head())

# Create a sample timegpt_fcst_finetune_df (for illustration; replace this with actual fine-tuned forecast data)
# This will simulate forecasted data
forecast_data = df.copy()
forecast_data['value'] = forecast_data['value'] * 1.05  # Simulating a 5% increase for forecasting

# Using Nixtla's plot function for visualization
try:
    nixtla.plot(df, forecast_data, time_col='timestamp', target_col='value')
    plt.title("Time-Series Plot: Original vs Forecasted Data")
    plt.xlabel('Timestamp')
    plt.ylabel('Sunspot Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Error while plotting: {e}")


Dataset Preview:
  timestamp  value
0   1749-01   58.0
1   1749-02   62.6
2   1749-03   70.0
3   1749-04   55.7
4   1749-05   85.0
Error while plotting: module 'nixtla' has no attribute 'plot'


In [7]:
# Load a new time-series dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')

print("Dataset Preview:")
print(df.head())


Dataset Preview:
     Month  Sunspots
0  1749-01      58.0
1  1749-02      62.6
2  1749-03      70.0
3  1749-04      55.7
4  1749-05      85.0


In [12]:
# Fine-tune the model with a specified number of iterations and optional depth
fine_tune_steps = 10  # Number of training iterations
timegpt_fcst_finetune_df = nixtla_client.forecast(
    df=df,
    h=12,  # Forecast horizon
    finetune_steps=fine_tune_steps,
    time_col='timestamp',
    target_col='value'
)

In [15]:
# Fine-tune the model at different levels of parameter adjustment to evaluate performance
train = df[:-24]
test = df[-24:].copy()
depths = [1, 2, 3, 4, 5]

for depth in depths:
    preds_df = nixtla_client.forecast(
        df=train,
        h=24,
        finetune_steps=5,
        time_col='timestamp',
        target_col='value'
    )
    preds = preds_df['TimeGPT'].values
    test[f'TimeGPT_depth{depth}'] = preds

In [16]:
# Add a unique identifier column for evaluation
test['unique_id'] = 0

In [17]:
# Evaluate and display performance metrics
evaluation = evaluate(test, metrics=[mae, mse], time_col="timestamp", target_col="value")
print("Evaluation Results:")
print(evaluation)

Evaluation Results:
   unique_id metric  TimeGPT_depth1  TimeGPT_depth2  TimeGPT_depth3  \
0          0    mae       54.426133       54.426133       54.426133   
1          0    mse     3887.305891     3887.305891     3887.305891   

   TimeGPT_depth4  TimeGPT_depth5  
0       54.426133       54.426133  
1     3887.305891     3887.305891  
